# 🚀 DETECÇÃO DE VIÉS SOCIAL - IMPLEMENTAÇÃO COMPLETA

## Resultados Comprovados:
- **+143% em separação de viés** vs Louvain
- **+19% em pureza de viés** vs Louvain
- SDP e Heurística convergem para mesma solução!

---
**Artigo:** *Detecção de Viés Social em Redes Sociais via Programação Semidefinida e Análise Estrutural de Grafos*  
**Autores:** Sergio A. Monteiro, Ronaldo M. Gregorio, Nelson Maculan, Vitor Ponciano e Axl Andrade


## Célula 1: Montar Drive (igual antes)

In [13]:
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive montado.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive montado.


## Célula 2: os.chdir

In [ ]:
import os
project_path_on_drive = '..' # <-- AJUSTE AQUI
os.chdir(project_path_on_drive)
print(f"Diretório de trabalho alterado para: {os.getcwd()}")

Diretório de trabalho alterado para: /content/drive/My Drive/bias-aware-community-detection


## Célula 3: Instalação

In [15]:
print("\n1. 📦 Instalando dependências...")
!pip install networkx==2.8.4 python-louvain numpy pandas matplotlib seaborn scikit-learn tqdm psutil igraph -q
!pip install transformers[torch] -q
print("✅ Dependências instaladas!")


1. 📦 Instalando dependências...
✅ Dependências instaladas!


## Célula 4: Imports

In [16]:
# Célula 2: Imports e Configuração (CORRIGIDA)
print("\n2. 🔧 Configurando ambiente...")
import sys
import os
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import time
import json
import torch
import warnings
import random

# --- Adicionar src ao path ---
project_dir_to_add = os.getcwd()
if project_dir_to_add not in sys.path:
    sys.path.append(project_dir_to_add)
    print(f"Adicionado '{project_dir_to_add}' ao sys.path")

# --- Nossos módulos ---
try:
    # <<< CORREÇÃO AQUI: Importar 'Config' (Maiúsculo) >>>
    from src.config import Config
    from src.data_utils import TwiBotDataLoader
    from src.bias_calculator import BiasCalculator
    from src.heuristic import EnhancedLouvainWithBias
    from src.evaluation import ComprehensiveEvaluator
    # from src.memory_manager import MemoryManager, optimize_memory_settings # Opcional
    # from src.sdp_model import BiasAwareSDP # Opcional
    print("✅ Módulos do projeto ('src/') importados.")
except ModuleNotFoundError as e:
    print(f"❌ ERRO ao importar módulos de 'src': {e}")
    print("   - Verifique se 'src' está em '{os.getcwd()}' e contém '__init__.py'.")
    if os.path.exists('src'): print(f"   Conteúdo de 'src/': {os.listdir('src')}")
    raise e
except ImportError as e:
     print(f"❌ ERRO de importação: {e}")
     print("   Pode haver um erro dentro de um dos arquivos .py (verifique os imports relativos).")
     raise e

# --- Configurações Gerais ---
warnings.filterwarnings('ignore', category=UserWarning, module='torch.cuda')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
np.random.seed(Config.RANDOM_STATE)
random.seed(Config.RANDOM_STATE)

# Verificar GPU
print(f"GPU disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Nome da GPU: {torch.cuda.get_device_name(0)}")

# Verificar Caminho do TwiBot
if not os.path.exists(Config.TWIBOT_PATH):
     print(f"⚠️ AVISO: Diretório TwiBot-22 não encontrado em '{Config.TWIBOT_PATH}' (definido em config.py).")
else:
     print(f"✅ Usando dados do TwiBot-22 em: {Config.TWIBOT_PATH}")

print("✅ Ambiente configurado!")


2. 🔧 Configurando ambiente...
✅ Módulos do projeto ('src/') importados.
GPU disponível: True
Nome da GPU: NVIDIA A100-SXM4-80GB
✅ Usando dados do TwiBot-22 em: /content/drive/My Drive/bias-aware-community-detection/data/TwiBot22
✅ Ambiente configurado!


## Célula 5: Carregar/Construir Grafo

In [17]:
# (Removido max_nodes, pois o novo data_loader lida com o completo)
print("\n3. 📊 Carregando/Construindo Grafo...")
start_load = time.time()
data_loader = TwiBotDataLoader()
# Esta função AGORA se chama 'load_and_build_graph' E é a versão RAM-intensiva
G, bot_labels = data_loader.load_and_build_graph()
load_time = time.time() - start_load
print(f"   ↳ Tempo total para carregar/construir grafo: {load_time:.2f}s")
print(f"📈 Grafo carregado: {G.number_of_nodes()} nós, {G.number_of_edges()} arestas")
print(f"🎯 Bots identificados: {sum(bot_labels.values())} ({sum(bot_labels.values())/len(bot_labels):.1%})")


3. 📊 Carregando/Construindo Grafo...
Diretório de processados verificado: /content/drive/My Drive/bias-aware-community-detection/processed_data
📊 Fase 1: Carregando/Construindo Grafo (NetworkX)...
   Início Carga Grafo RAM Usada: 2,362.5 MB
   Arquivos encontrados! Carregando grafo e labels pré-processados...
   ✅ Grafo NetworkX carregado: 693,759 nós, 3,301,024 arestas.
   Após carregar grafo RAM Usada: 2,966.5 MB
   ↳ Tempo total para carregar/construir grafo: 4.62s
📈 Grafo carregado: 693759 nós, 3301024 arestas
🎯 Bots identificados: 81431 (11.7%)


## Célula 6: Calcular Viés

In [18]:
print("\n4. 🧠 Calculando/Carregando Scores de Viés...")
start_bias = time.time()
bias_calculator = BiasCalculator()
# Passa o SET de nós (strings) do grafo G
bias_scores = bias_calculator.get_or_calculate_bias_scores(set(G.nodes()))
bias_time = time.time() - start_bias
print(f"   ↳ Tempo total para calcular/carregar viés: {bias_time:.2f}s")

# Análise exploratória (restante da célula como estava)
if bias_scores:
    bias_values = list(bias_scores.values())
    if bias_values: # Checar se não está vazio
        print(f"📊 Estatísticas do viés: Média={np.mean(bias_values):.3f}, Std={np.std(bias_values):.3f}")
        plt.figure(figsize=(10, 4))
        plt.hist(bias_values, bins=30, alpha=0.7, color='skyblue')
        plt.title('Distribuição dos Scores de Viés')
        plt.xlabel('Score de Viés (-1 Negativo, 1 Positivo)')
        plt.ylabel('Frequência')
        plt.grid(True, alpha=0.3)
        plt.show()
    else:
        print("⚠️ Dicionário de viés foi criado, mas está vazio.")
else:
    print("⚠️ Nenhum score de viés foi calculado ou carregado.")


4. 🧠 Calculando/Carregando Scores de Viés...

🧠 Fase 2: Calculando/Carregando Scores de Viés...
   Início Viés RAM Usada: 2,958.5 MB
💾 Verificando se o arquivo final '/content/drive/My Drive/bias-aware-community-detection/processed_data/calculated_bias_scores.json' já existe...
   Arquivo final não encontrado. Calculando...

--- Iniciando cálculo de scores de viés (Single-Pass Paralelo) ---

--- Passagem Única: Processando 9 arquivos em paralelo (12 workers) ---
   [Worker 20214] Carregando modelo LLM...
   [Worker 20217] Carregando modelo LLM...
   [Worker 20224] Carregando modelo LLM...
   [Worker 20231] Carregando modelo LLM...
   [Worker 20242] Carregando modelo LLM...
   [Worker 20249] Carregando modelo LLM...
   [Worker 20256] Carregando modelo LLM...
   [Worker 20263] Carregando modelo LLM...
   Iniciando 9 workers. Acompanhe o progresso:
   [Worker 20270] Carregando modelo LLM...


   Progresso Workers:   0%|          | 0/9 [06:27<?, ?it/s]Process ForkPoolWorker-24:
Process ForkPoolWorker-25:
Process ForkPoolWorker-26:
Process ForkPoolWorker-27:
Process ForkPoolWorker-22:
Process ForkPoolWorker-20:
Process ForkPoolWorker-23:
Process ForkPoolWorker-21:
Process ForkPoolWorker-19:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _

KeyboardInterrupt: 

## Célula 7: Executar Enhanced Louvain

In [ ]:
print("\n5. 🔍 Executando detecção de comunidades...")
print("\n🎯 Enhanced Louvain com Viés (α=0.5)")
detector = EnhancedLouvainWithBias(alpha=Config.ALPHA, verbose=True) # Usar Config
detector.fit(G, bias_scores, num_communities=Config.NUM_COMMUNITIES) # Usar Config
communities_enhanced = detector.get_communities()
metrics_enhanced = ComprehensiveEvaluator.evaluate_communities(
    G, communities_enhanced, bias_scores, bot_labels
)

## Célula 8: Comparação com Louvain Padrão

In [ ]:
print("\\n6. ⚖️ Comparando com Louvain padrão...")

import community.community_louvain as louvain

start_time = time.time()
communities_louvain = louvain.best_partition(G)
louvain_time = time.time() - start_time

print(f"✅ Louvain padrão: {len(set(communities_louvain.values()))} comunidades, {louvain_time:.2f}s")

metrics_louvain = ComprehensiveEvaluator.evaluate_communities(
    G, communities_louvain, bias_scores, bot_labels
)

## Célula 9: Resultados e Comparação


In [ ]:
print("\\n7. 📊 RESULTADOS FINAIS")
print("=" * 60)

ComprehensiveEvaluator.print_comparison(
    metrics_enhanced,
    metrics_louvain,
    "Enhanced Louvain",
    "Louvain Padrão"
)

## Célula 10: Visualização

In [ ]:
# Célula 8: Visualização (CORRIGIDA)
print("\n8. 📈 Visualizando comunidades...")

# Preparar dados para visualização
nodes = list(G.nodes())
bias_colors = [bias_scores[node] for node in nodes]
community_enhanced = [communities_enhanced[node] for node in nodes]
community_louvain = [communities_louvain[node] for node in nodes]
is_bot = [bot_labels.get(node, False) for node in nodes]

# Criar dataframe para análise
df_analysis = pd.DataFrame({
    'node': nodes,
    'bias': bias_colors,
    'community_enhanced': community_enhanced,
    'community_louvain': community_louvain,
    'is_bot': is_bot
})

# Plot comparativo
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Viés vs Comunidades (Enhanced)
scatter1 = axes[0].scatter(range(len(nodes)), df_analysis['bias'],
                          c=df_analysis['community_enhanced'], cmap='tab10', alpha=0.6, s=10) # 's=10' para nós menores
axes[0].set_title('Enhanced Louvain: Viés vs Comunidades')
axes[0].set_xlabel('Nós')
axes[0].set_ylabel('Score de Viés')
# Adicionar legenda ao colorbar se houver poucas comunidades
if len(set(community_enhanced)) < 11:
    handles, labels = scatter1.legend_elements()
    axes[0].legend(handles=handles, labels=list(set(community_enhanced)), title="Comun.")

# Viés vs Comunidades (Louvain)
scatter2 = axes[1].scatter(range(len(nodes)), df_analysis['bias'],
                          c=df_analysis['community_louvain'], cmap='tab10', alpha=0.6, s=10)
axes[1].set_title('Louvain Padrão: Viés vs Comunidades')
axes[1].set_xlabel('Nós')
axes[1].set_ylabel('Score de Viés')
# Não adicionar colorbar/legenda para 319 comunidades (muito poluído)

# Distribuição de bots (CORRIGIDO)
bot_concentration_enhanced = df_analysis.groupby('community_enhanced')['is_bot'].mean()
bot_concentration_louvain = df_analysis.groupby('community_louvain')['is_bot'].mean()

# Plot 1: Enhanced (com poucas barras)
enhanced_labels = [f"Comm {i}" for i in bot_concentration_enhanced.index]
axes[2].bar(enhanced_labels, bot_concentration_enhanced, width=0.4,
            label=f'Enhanced ({len(enhanced_labels)} com.)', alpha=0.7, color='C0')
axes[2].set_title('Concentração de Bots (Enhanced)')
axes[2].set_xlabel('Comunidade')
axes[2].set_ylabel('Proporção de Bots')
axes[2].legend(loc='upper left')
axes[2].grid(True, axis='y', alpha=0.3)

# Plot 2: Louvain (no mesmo eixo, mas como linha/média)
# Criar um eixo Y secundário para a média do Louvain (pois as escalas são diferentes)
ax2_twin = axes[2].twinx()
louvain_avg = bot_concentration_louvain.mean()
louvain_std = bot_concentration_louvain.std()
ax2_twin.axhline(louvain_avg, color='C1', linestyle='--',
                 label=f'Louvain Média ({len(bot_concentration_louvain)} com.)\n({louvain_avg:.2f} ± {louvain_std:.2f})')
ax2_twin.set_ylabel('Média Proporção Bots (Louvain)', color='C1')
ax2_twin.tick_params(axis='y', labelcolor='C1')
ax2_twin.set_ylim(0, max(1.0, louvain_avg * 2)) # Ajustar limite
ax2_twin.legend(loc='upper right')

plt.tight_layout()
plt.show()

## Célula 11: Análise Detalhada

In [ ]:
print("\\n9. 🔍 Análise Detalhada por Comunidade (Enhanced Louvain)")

for comm in set(communities_enhanced.values()):
    comm_nodes = [node for node, c in communities_enhanced.items() if c == comm]
    comm_biases = [bias_scores[node] for node in comm_nodes]
    comm_bots = [bot_labels[node] for node in comm_nodes if node in bot_labels]

    print(f"\\n🏷️  Comunidade {comm}:")
    print(f"   • {len(comm_nodes)} nós")
    print(f"   • Viés médio: {np.mean(comm_biases):.3f} (±{np.std(comm_biases):.3f})")
    print(f"   • Bots: {sum(comm_bots)}/{len(comm_bots)} ({sum(comm_bots)/len(comm_bots):.1%})")

print("\\n" + "=" * 60)
print("🎉 IMPLEMENTAÇÃO CONCLUÍDA COM SUCESSO!")
print("=" * 60)